In [55]:
import warnings
import os
# Suppress all DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from google.genai import types
from google.adk.agents.llm_agent import LlmAgent, Agent
from google.adk.runners import Runner
from toolbox_core import ToolboxSyncClient
from google.adk.sessions import InMemorySessionService
from mem0 import MemoryClient
import nest_asyncio
import asyncio
from dotenv import load_dotenv


In [56]:
load_dotenv()
nest_asyncio.apply()

APP_NAME = "agents"
USER_ID = "testing"

mem0 = MemoryClient(api_key="m0-H0GaPxc7aXWQQNtzTIXdtrjjmEGfeOS5L99PKtg3")


toolbox_client = ToolboxSyncClient(
    url="http://127.0.0.1:5000"
)
database_tools = toolbox_client.load_toolset("cs_agent_tools")

In [57]:
# Define memory function tools
def search_memory(query: str, user_id: str = USER_ID) -> dict:
    """Search through past conversations and memories to find relevant information about the user or previous interactions.
    
    Use this tool when:
    - The user asks about something that might have been discussed before
    - You need to recall user preferences, past orders, or personal information
    - The user references something from a previous conversation
    
    Args:
        query: The search query to find relevant memories (e.g., "user's name", "previous orders", "preferences")
        user_id: The user's unique identifier (defaults to current session user)
    
    Returns:
        A dictionary with status and either memories (formatted context) or a message if no memories found
    """
    # For Platform API, user_id goes in filters
    filters = {"user_id": user_id}
    memories = mem0.search(query, filters=filters)
    if memories.get('results', []):
        memory_list = memories['results']
        memory_context = "\n".join([f"- {mem['memory']}" for mem in memory_list])
        return memory_context
    return {"status": "no_memories", "message": "No relevant memories found"}

def save_memory(content: str, user_id: str = USER_ID) -> dict:
    """Save important information to memory for future reference.
    
    Use this tool when:
    - The user provides personal information (name, preferences, contact details)
    - The user mentions something important that should be remembered
    - You want to store context about the user's situation or needs
    
    Args:
        content: The information to save (e.g., "User prefers express shipping", "User's email is alice@example.com")
        user_id: The user's unique identifier (defaults to current session user)
    
    Returns:
        A dictionary with status and confirmation message
    """
    try:
        result = mem0.add([{"role": "user", "content": content}], user_id=user_id)
        return result
    except Exception as e:
        return {"status": "error", "message": f"Failed to save memory: {str(e)}"}


In [75]:
PROMPT_INSTRUCTION = """
You are a professional customer support agent for an online shop. 
Always greet the customer courteously and try to resolve their questions or concerns efficiently.

## Tool Usage Guidelines:

1. **Search Memory:**
   - ALWAYS Use the search_memory function to recall past conversations and user preferences using the 
   USER_ID: {USER_ID}.

2. **Check Order Status (Specific Order):**
   - If a customer asks about a specific order (status, shipment, or delivery), use the 'get-order-status' tool.
   - **Input Format:** The tool requires a **numeric** Order ID (e.g., 1, 5, 20). 
   - **Normalization:** If the user says "Order #5", "Number 5", or "Order 5", strictly extract just the integer `5` for the tool argument.
   - If the customer does not provide an Order ID, politely ask for the Order Number.

3. **Check Order History (All Orders):**
   - If a customer wants to know all their orders or their history, use the 'find-customer-orders' tool.
   - **Input Format:** The tool requires a valid **Email Address** (e.g., alice@example.com).
   - If the user hasn't provided their email address yet, ask them for it politely to look up their account. Do not ask for a "Customer ID", ask for their "Email".

**Response Guidelines:**
- The database returns 'items' as a raw JSON list (e.g., `[{{"product": "Mouse", "qty": 1...}}]`). Do not show raw JSON to the user. Parse it and describe the items naturally (e.g., "You ordered 1 Wireless Mouse").
- Summarize information clearly (Status, Date, and Total Amount).
- If you cannot answer a question or the data is missing, apologize and suggest contacting human support.
"""

PROMPT_INSTRUCTION = PROMPT_INSTRUCTION.format(USER_ID=USER_ID)

In [76]:
# Combine database tools and memory tools
root_agent = Agent(
    model='gemini-2.5-flash',
    name='customer_support_assistant',
    description=(
        'An expert customer support agent helping users with order-related questions and requests. '
        'Provides fast, clear, and friendly assistance with memory of past interactions.'
    ),
    instruction=PROMPT_INSTRUCTION,
    tools= [*database_tools, search_memory],
)

In [77]:
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

In [78]:
async def main():

    messages = []
    # Create a new session
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=f"session_{USER_ID}")

    while True:

        print("=" * 80)
        user_input = input("You: ")
        print("User: ", user_input)
        if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
            break
        messages.append({"role": "user", "content": user_input})
        content = types.Content(role='user', parts=[types.Part(text=user_input)])
        response = runner.run(user_id=USER_ID, session_id=f"session_{USER_ID}", new_message=content)
        
        for event in response:
            # Print final response
            if event.is_final_response() and event.content:
                print("Agent: ", event.content.parts[0].text)
                messages.append({"role": "assistant", "content": event.content.parts[0].text})
        
        # print(messages)
        save_memory(messages, USER_ID)


if __name__ == "__main__":
    asyncio.run(main())

User:  Hi


Task was destroyed but it is pending!
task: <Task pending name='Task-64' coro=<_wait_for_close() running at d:\CSA\.venv\Lib\site-packages\aiohttp\connector.py:136>>
C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\base_events.py:750: RuntimeWarning: coroutine '_wait_for_close' was never awaited
  self._ready.clear()


Agent:  Hello Ali Zain! It's great to have you back. I see you've previously inquired about orders #20 and #18.

How can I help you today? Do you have another question about an order, or is there something else I can assist you with?


HTTP error occurred: Client error '400 Bad Request' for url 'https://api.mem0.ai/v1/memories/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400


User:  I wanna know about the order # 2
Agent:  I can help you with that!

Here are the details for Order #2:
*   **Status:** DELIVERED
*   **Order Date:** 2025-08-23
*   **Items:**
    *   1 Wireless Mouse for $25
    *   1 Mouse Pad for $10
*   **Total Amount:** $35.00

Is there anything else I can assist you with regarding this order or any other queries you might have?


HTTP error occurred: Client error '400 Bad Request' for url 'https://api.mem0.ai/v1/memories/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400


User:  quit


### Multi Agent Architecture

In [42]:
from google.genai import types
from google.adk.agents.llm_agent import LlmAgent, Agent
from google.adk.runners import Runner
from google.adk.tools import FunctionTool

from toolbox_core import ToolboxSyncClient
from google.adk.sessions import InMemorySessionService
from dotenv import load_dotenv
# from memory import search_memory, save_memory


load_dotenv()

APP_NAME = "agents"
USER_ID = "1234"

toolbox_client = ToolboxSyncClient(
    url="http://127.0.0.1:5000"
)

database_tools = toolbox_client.load_toolset("cs_agent_tools")

mem0 = MemoryClient(api_key=os.getenv("MEM0_API_KEY"))

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000021BC095DD10>


In [45]:
def search_memory(query: str, user_id: str) -> dict:
    """Search through past conversations and memories"""
    # For Platform API, user_id goes in filters
    filters = {"user_id": user_id}
    memories = mem0.search(query, filters=filters)
    if memories.get('results', []):
        memory_list = memories['results']
        memory_context = "\n".join([f"- {mem['memory']}" for mem in memory_list])
        return {"status": "success", "memories": memory_context}
    return {"status": "no_memories", "message": "No relevant memories found"}


In [52]:
memory_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='memory_search_agent',
    description=(
        'A specialized agent that searches through long-term memory to find relevant information '
        'from past conversations, user preferences, and previous interactions. '
        'Takes user input, searches memory, and returns formatted memory context for use by other agents.'
    ),
    instruction="""You are a memory search specialist. Your task is to:
1. Take the user's query and search through long-term memory using the search_memory tool
2. Analyze the search results to extract the most relevant information
3. Format and return the relevant memory context in a clear, structured way that provides useful context about:
   - User preferences and personal information
   - Past orders and interactions
   - Previous conversation topics
   - Any other relevant context that would help a customer support agent assist the user

Always use the search_memory tool with the user's query and user_id to retrieve relevant information.
If no relevant memories are found, return "No relevant memories found."

Format your response as a clear summary of relevant context that can be directly used by a customer support agent.""",
    tools=[search_memory],
)

In [ ]:
# Customer support agent - receives memory context and user input, retrieves data from database
customer_support_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='customer_support_assistant',
    description=(
        'An expert customer support agent helping users with order-related questions and requests. '
        'Receives memory context from the memory search agent and uses database tools to retrieve information.'
    ),
    instruction=f"""
You are a professional customer support agent for an online shop. 
Always greet the customer courteously and try to resolve their questions or concerns efficiently.

**Tool Usage Guidelines:**


1. **Check Order Status (Specific Order):**
   - If a customer asks about a specific order (status, shipment, or delivery), use the 'get-order-status' tool.
   - **Input Format:** The tool requires a **numeric** Order ID (e.g., 1, 5, 20). 
   - **Normalization:** If the user says "Order #5", "Number 5", or "Order 5", strictly extract just the integer `5` for the tool argument.
   - If the customer does not provide an Order ID, politely ask for the Order Number.

2. **Check Order History (All Orders):**
   - If a customer wants to know all their orders or their history, use the 'find-customer-orders' tool.
   - **Input Format:** The tool requires a valid **Email Address** (e.g., alice@example.com).
   - If the user hasn't provided their email address yet, ask them for it politely to look up their account. Do not ask for a "Customer ID", ask for their "Email".

**Response Guidelines:**
- The database returns 'items' as a raw JSON list (e.g., `[{{"product": "Mouse", "qty": 1...}}]`). Do not show raw JSON to the user. Parse it and describe the items naturally (e.g., "You ordered 1 Wireless Mouse").
- Summarize information clearly (Status, Date, and Total Amount).
- If you cannot answer a question or the data is missing, apologize and suggest contacting human support.
""",
    tools=database_tools,
)

In [56]:
from google.adk.tools.agent_tool import AgentTool

orchestrator = LlmAgent(
    model='gemini-2.5-flash',
    name='orchestrator',
    description="An orchestrator that coordinates the memory search and customer support agents.",
    instruction="""You are an orchestrator that coordinates the memory search and customer support agents.
    """,
    tools=[
        AgentTool(
            agent=memory_agent,
        ),
        AgentTool(
            agent=customer_support_agent,
        )
    ]
)

In [57]:
session_service = InMemorySessionService()
runner = Runner(agent=orchestrator, app_name=APP_NAME, session_service=session_service)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000021BC09DAD50>


In [ ]:
async def main():

    # Create a new session
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=f"session_{USER_ID}")

    while True:

        print("=" * 80)
        user_input = input("You: ")
        print("User: ", user_input)
        if user_input.lower() in ['quit', 'exit', 'bye', 'q']:
            break
        
        content = types.Content(role='user', parts=[types.Part(text=user_input)])
        response = runner.run(user_id=USER_ID, session_id=f"session_{USER_ID}", new_message=content)
        
        # Process events - Google ADK handles function calls automatically
        # Memory tools (search_memory, save_memory) will be called by the agent when needed
        for event in response:
            # Print final response
            if event.is_final_response() and event.content:
                print("Agent: ", event.content.parts[0].text)


if __name__ == "__main__":
    asyncio.run(main())

User:  Hi


Exception in thread Thread-60 (_asyncio_thread_main):
Traceback (most recent call last):
  File "d:\CSA\.venv\Lib\site-packages\aiohttp\connector.py", line 1532, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\CSA\.venv\Lib\site-packages\aiohttp\connector.py", line 1148, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\futures.py", line 286, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\tasks.py", line 375, in __wakeup
    future.result()
    ~~~~~~~~~~~~~^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\futures.

User:  


Exception in thread Thread-61 (_asyncio_thread_main):
Traceback (most recent call last):
  File "d:\CSA\.venv\Lib\site-packages\aiohttp\connector.py", line 1532, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\CSA\.venv\Lib\site-packages\aiohttp\connector.py", line 1148, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\futures.py", line 286, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\tasks.py", line 375, in __wakeup
    future.result()
    ~~~~~~~~~~~~~^^
  File "C:\Users\Ali Zain\AppData\Roaming\uv\python\cpython-3.13.8-windows-x86_64-none\Lib\asyncio\futures.